# AG2 Tutorial: Building a Financial Compliance Assistant

Welcome to this walkthrough on building a Financial Compliance Assistant using AG2 (AutoGen v2)! This tutorial is designed to guide you step-by-step, covering key AG2 concepts. We'll build a multi-agent system capable of processing transactions, flagging suspicious ones for human review, checking for duplicates, and generating structured summary reports.

## 1. Setup

First, let's install AG2 with OpenAI support and import all necessary libraries. We'll also set up our OpenAI API key using Colab's `userdata` feature for security.

In [ ]:
!pip install "ag2[openai]" -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.5/787.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
import os
import random
import json
from typing import Annotated, Any, List, Dict # Python 3.8 requires Dict, Python 3.9+ can use dict
from datetime import datetime, timedelta
from pydantic import BaseModel

from google.colab import userdata

from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

# Set up your OpenAI API Key
# Make sure you have an OPENAI_API_KEY secret in Colab
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

if OPENAI_API_KEY is None:
    print("OPENAI_API_KEY not found in Colab secrets. Please add it.")
else:
    # For AG2, it's good practice to set it as an environment variable if the LLMConfig expects it,
    # or pass it directly to LLMConfig.
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    print("OPENAI_API_KEY configured.")

OPENAI_API_KEY configured.


## 2. LLM Configuration

The `LLMConfig` is essential as it powers your agents' intelligence. It defines how agents connect to Large Language Models (LLMs), specifying the provider, model, authentication, and behavior parameters.

AG2 supports various providers (OpenAI, Gemini, Anthropic, etc.). For this tutorial, we'll use OpenAI's `gpt-4o-mini` model as specified in the documentation.

**Important**: Never hard-code API keys. We're using Colab's `userdata` and environment variables.

In [ ]:
llm_config_openai = LLMConfig(
    api_type="openai",                      # The provider
    model="gpt-4o-mini",                    # The specific model
    api_key=os.environ.get("OPENAI_API_KEY"),   # Authentication from environment
    temperature=0.2                         # Lower temperature for more consistent financial analysis
)

print("LLM Configuration for OpenAI created successfully.")

LLM Configuration for OpenAI created successfully.


### Integrating LLM Configuration with Agents

You can apply this `LLMConfig` to agents either by passing it as a keyword argument during agent creation or by using a context manager. The context manager applies the configuration to all agents created within its scope.

In [ ]:
# Example of using the context manager (we'll use this approach later)
with llm_config_openai:
    # Any agent created here would use llm_config_openai by default
    # e.g., agent = ConversableAgent(name="sample_agent_in_context")
    pass

print("Context manager for LLMConfig demonstrated (no agents created in this example cell).")

## 3. ConversableAgent: Building Intelligent Agents

The `ConversableAgent` is the core building block in AG2. It acts as the brain and personality of your AI system. Powered by an `LLMConfig`, it can communicate, process information, follow instructions, and execute tools.

Key parameters for `ConversableAgent`:
- `name`: A unique identifier.
- `system_message`: Instructions defining its role, personality, and behavior.
- `llm_config`: The LLM configuration.

In [ ]:
# Create the agent using the LLM configuration directly
helpful_agent = ConversableAgent(
    name="helpful_agent",
    system_message="You are a helpful AI assistant that provides concise answers.",
    llm_config=llm_config_openai
)

print(f"Agent '{helpful_agent.name}' created.")

Agent 'helpful_agent' created.


### Interacting with a ConversableAgent

We use `run()` to establish a workflow (it returns an iterator) and `process()` to execute it and see console output. `process()` simulates a chat-like experience.

In [ ]:
# Establish the workflow
response_iterator = helpful_agent.run(
    message="What's the capital of France?",
    max_turns=1,  # Limit conversation length for this simple query
    user_input=False # No user input needed for this example after the initial message
)

# Process the workflow
response_iterator.process()

user (to helpful_agent):

What's the capital of France?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
helpful_agent (to user):

The capital of France is Paris.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (f3eb853f-5971-45d4-ae7c-a8231e2396ad): Maximum turns (1) reached


### Financial Compliance Example: Basic Agent
Let's create a simple financial agent to answer a question.

In [ ]:
# Create a basic financial agent using the context manager for LLM config
with llm_config_openai:
    finance_agent_basic = ConversableAgent(
        name="finance_agent_basic",
        system_message="You are a financial assistant who helps analyze financial data and transactions."
    )

# Run the agent with a prompt
response_iterator_finance = finance_agent_basic.run(
    message="Can you explain what makes a transaction suspicious?",
    max_turns=1 # Limit to one turn for the explanation
)

# Iterate through the chat automatically with console output
response_iterator_finance.process()

user (to finance_agent_basic):

Can you explain what makes a transaction suspicious?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
finance_agent_basic (to user):

Certainly! A transaction may be considered suspicious for several reasons, often related to patterns or behaviors that deviate from the norm. Here are some common indicators:

1. **Unusual Amounts**: Transactions that are significantly larger or smaller than typical for the account holder can raise red flags.

2. **Inconsistent Patterns**: A sudden change in transaction frequency or volume, such as a spike in activity after a period of inactivity, can be suspicious.

3. **Geographic Anomalies**: Transactions originating from or being sent to high-risk countries or regions, especially if they are inconsistent with the customer’s known behavior.

4. **Structuring or Smurfing**: Breaking down large transactions into smaller amounts to avoid detection or reporting 

## 4. Human in the Loop (HITL): Adding Human Oversight

HITL allows agents to collaborate with humans at critical decision points. This is crucial for tasks like financial compliance where human judgment is essential.

We implement HITL using the `human_input_mode` parameter in `ConversableAgent`:
- `ALWAYS`: Agent always prompts for human input.
- `TERMINATE`: Agent asks for input only when terminating.
- `NEVER`: Agent never asks for human input.

Let's build a system where a `finance_bot` processes transactions and asks a `human` agent for approval on suspicious ones.

In [ ]:
# Define the system message for our finance bot
finance_system_message_hitl = """
You are a financial compliance assistant. You will be given a set of transaction descriptions.
For each transaction:
- If it seems suspicious (e.g., amount > $10,000, vendor is unusual, memo is vague), ask the human agent for approval.
- Otherwise, approve it automatically.
Provide the full set of transactions to approve at one time.
If the human gives a general approval, it applies to all transactions requiring approval.
When all transactions are processed, summarize the results and say "You can type exit to finish".
"""

# Create the finance agent with LLM intelligence using the context manager
with llm_config_openai:
    finance_bot_hitl = ConversableAgent(
        name="finance_bot_hitl",
        system_message=finance_system_message_hitl,
    )

# Create the human agent for oversight
human_agent_hitl = ConversableAgent(
    name="human_hitl",
    human_input_mode="ALWAYS",  # Always ask for human input
)

# Generate sample transactions
VENDORS = ["Staples", "Acme Corp", "CyberSins Ltd", "Initech", "Globex", "Unicorn LLC"]
MEMOS = ["Quarterly supplies", "Confidential", "NDA services", "Routine payment", "Urgent request", "Reimbursement"]

def generate_transaction():
    amount = random.choice([500, 1500, 9999, 12000, 23000, 4000])
    vendor = random.choice(VENDORS)
    memo = random.choice(MEMOS)
    return f"Transaction: ${amount} to {vendor}. Memo: {memo}."

transactions_hitl = [generate_transaction() for _ in range(3)]

initial_prompt_hitl = (
    "Please process the following transactions:\n\n" +
    "\n".join([f"{i+1}. {tx}" for i, tx in enumerate(transactions_hitl)])
)

print("--- Initial Prompt for HITL ---")
print(initial_prompt_hitl)
print("-------------------------------")

# Start the conversation from the human agent
# Note: In a real Colab session, this will prompt for input in the output cell.
# For automated runs or non-interactive environments, you might need a different setup for human_input_mode.
response_iterator_hitl = human_agent_hitl.run(
    recipient=finance_bot_hitl,
    message=initial_prompt_hitl,
    max_turns=3 # Allow a few turns for request, human approval, and summary
)

# Display the response
response_iterator_hitl.process()

--- Initial Prompt for HITL ---
Please process the following transactions:

1. Transaction: $9999 to CyberSins Ltd. Memo: NDA services.
2. Transaction: $4000 to Unicorn LLC. Memo: Reimbursement.
3. Transaction: $4000 to CyberSins Ltd. Memo: Quarterly supplies.
-------------------------------
human_hitl (to finance_bot_hitl):

Please process the following transactions:

1. Transaction: $9999 to CyberSins Ltd. Memo: NDA services.
2. Transaction: $4000 to Unicorn LLC. Memo: Reimbursement.
3. Transaction: $4000 to CyberSins Ltd. Memo: Quarterly supplies.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
finance_bot_hitl (to human_hitl):

Here are the transactions requiring approval:

1. Transaction: $9999 to CyberSins Ltd. Memo: NDA services. (Approved automatically)
2. Transaction: $4000 to Unicorn LLC. Memo: Reimbursement. (Approved automatically)
3. Transaction: $4000 to CyberSins Ltd. Memo: Quarterly supplies. (Approved auto

**Running the HITL Example:**
When you run the cell above, the `finance_bot_hitl` will analyze transactions. If any are suspicious, it will pause and the output cell will display:
```
Replying as human_hitl. Provide feedback to finance_bot_hitl. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:
```
You'll need to type your response (e.g., `approve`, `deny all suspicious transactions`, or `exit`) and press Enter.

## 5. Agent Orchestration: Coordinating Multiple Agents (Group Chat)

For more complex workflows, we need specialized agents collaborating. The Group Chat pattern in AG2 allows this. We'll use `AutoPattern`, where a group manager agent automatically selects the next speaker based on conversation context.

Let's add a `summary_bot` to our financial compliance system to generate formatted reports after `finance_bot` processes transactions.

### Automatic Termination in Group Chats
We'll also implement automatic termination. The `summary_bot` will add a special marker (`==== SUMMARY GENERATED ====`) to its output, and we'll define a function `is_termination_msg` to detect this marker, ending the chat automatically.

In [ ]:
# Define the system message for our finance bot (can reuse or adapt)
finance_system_message_group = """
You are a financial compliance assistant. You will be given a set of transaction descriptions.
For each transaction:
- If it seems suspicious (e.g., amount > $10,000, vendor is unusual, memo is vague), ask the human agent for approval.
- Otherwise, approve it automatically.
Provide the full set of transactions to approve at one time.
If the human gives a general approval, it applies to all transactions requiring approval.
When all transactions are processed, clearly state that all transactions are processed and summarize the results. Then say 'Handing over for final report.' to signal completion to the group manager.
"""

# Define the system message for the summary agent
summary_system_message_group = """
You are a financial summary assistant. You will be given a set of transaction details and their approval status from the finance bot.
Your task is to summarize the results of the transactions processed by the finance bot.
Generate a markdown table with the following columns:
- Vendor
- Memo
- Amount
- Status (Approved/Rejected)
The summary should include the total number of transactions, the number of approved transactions, and the number of rejected transactions.
The summary should be concise and clear.
Once you've generated the summary, append the following marker on a new line:
==== SUMMARY GENERATED ====
"""

# Create agents using the context manager for the shared LLM config
with llm_config_openai:
    finance_bot_group = ConversableAgent(
        name="finance_bot_group",
        system_message=finance_system_message_group,
    )
    summary_bot_group = ConversableAgent(
        name="summary_bot_group",
        system_message=summary_system_message_group,
    )

human_agent_group = ConversableAgent(
    name="human_group",
    human_input_mode="ALWAYS",
)

def is_termination_msg_group(msg: dict[str, Any]) -> bool:
    content = msg.get("content", "")
    return (content is not None) and "==== SUMMARY GENERATED ====" in content

# Generate sample transactions
transactions_group = [generate_transaction() for _ in range(3)]
initial_prompt_group = (
    "Please process the following transactions:\n\n" +
    "\n".join([f"{i+1}. {tx}" for i, tx in enumerate(transactions_group)])
)

print("--- Initial Prompt for Group Chat ---")
print(initial_prompt_group)
print("-----------------------------------")

# Create pattern for the group chat
pattern_group = AutoPattern(
    initial_agent=finance_bot_group,                   # Start with the finance bot
    agents=[finance_bot_group, summary_bot_group],     # All AI agents in the group chat
    user_agent=human_agent_group,                      # Human-in-the-loop agent
    group_manager_args={
        "llm_config": llm_config_openai,          # Config for group manager's decision making
        "is_termination_msg": is_termination_msg_group # Add termination condition
    }
)

# Initialize the group chat
result_group, _, _ = initiate_group_chat(
    pattern=pattern_group,
    messages=initial_prompt_group,                     # Initial request with transactions
)

print("\n--- Group Chat Summary ---")
print(result_group.summary)
print("\n--- Last Message ---")
if result_group.chat_history:
    print(result_group.chat_history[-1]["content"])
else:
    print("No chat history recorded.")

--- Initial Prompt for Group Chat ---
Please process the following transactions:

1. Transaction: $23000 to CyberSins Ltd. Memo: Routine payment.
2. Transaction: $4000 to Initech. Memo: Reimbursement.
3. Transaction: $23000 to CyberSins Ltd. Memo: Confidential.
-----------------------------------
human_group (to chat_manager):

Please process the following transactions:

1. Transaction: $23000 to CyberSins Ltd. Memo: Routine payment.
2. Transaction: $4000 to Initech. Memo: Reimbursement.
3. Transaction: $23000 to CyberSins Ltd. Memo: Confidential.

--------------------------------------------------------------------------------

Next speaker: finance_bot_group


>>>>>>>> USING AUTO REPLY...
finance_bot_group (to chat_manager):

The following transactions require approval due to their suspicious nature:

1. Transaction: $23000 to CyberSins Ltd. Memo: Routine payment.
2. Transaction: $23000 to CyberSins Ltd. Memo: Confidential.

Please provide approval for these transactions. The transac

**Running the Group Chat Example:**
Similar to the HITL example, if `finance_bot_group` flags transactions, you'll be prompted for input as `human_group`. After your approval, `finance_bot_group` should summarize and indicate a handoff. Then, `summary_bot_group` should generate its report with the `==== SUMMARY GENERATED ====` marker, causing the chat to terminate automatically.

## 6. Tools: Extending Agent Capabilities

Tools allow agents to interact with the external world or perform specialized functions beyond their LLM's built-in knowledge. We'll add a `check_duplicate_payment` tool to our `finance_bot`.

Tool usage in AG2 involves:
1.  **Selection**: An agent (LLM) decides which tool to use.
2.  **Execution**: An executor (often internal to AG2's group chat or agent runtime) invokes the tool and returns results.

In [ ]:
# Mock database of previous transactions
def get_previous_transactions() -> list[dict[str, Any]]:
    today = datetime.now()
    return [
        {
            "vendor": "Staples",
            "amount": 500.00,
            "date": (today - timedelta(days=3)).strftime("%Y-%m-%d"),
            "memo": "Quarterly supplies",
        },
        {
            "vendor": "Acme Corp",
            "amount": 1500.00,
            "date": (today - timedelta(days=10)).strftime("%Y-%m-%d"),
            "memo": "NDA services",
        },
        {
            "vendor": "Globex",
            "amount": 12000.00,
            "date": (today - timedelta(days=5)).strftime("%Y-%m-%d"),
            "memo": "Confidential",
        },
    ]

# Simple duplicate detection function (tool)
def check_duplicate_payment(
    vendor: Annotated[str, "The vendor name"],
    amount: Annotated[float, "The transaction amount"],
    memo: Annotated[str, "The transaction memo"]
) -> dict[str, Any]:
    """Check if a transaction appears to be a duplicate of a recent payment (within 7 days)."""
    previous_transactions = get_previous_transactions()
    today = datetime.now()

    for tx in previous_transactions:
        tx_date = datetime.strptime(tx["date"], "%Y-%m-%d")
        date_diff = (today - tx_date).days

        if (
            tx["vendor"] == vendor and
            tx["memo"] == memo and
            abs(tx["amount"] - amount) < 0.01 and # Compare floats with tolerance
            date_diff <= 7
        ):
            return {
                "is_duplicate": True,
                "reason": f"Duplicate payment to {vendor} for ${amount:.2f} on {tx['date']}"
            }

    return {
        "is_duplicate": False,
        "reason": "No recent duplicates found"
    }

print("Duplicate detection tool 'check_duplicate_payment' defined.")

Duplicate detection tool 'check_duplicate_payment' defined.


In [ ]:
# Define the system message for our finance bot with tool usage instructions
finance_system_message_tools = """
You are a financial compliance assistant. You will be given a set of transaction descriptions.

For each transaction:
1. First, extract the vendor name, amount, and memo.
2. Check if the transaction is a duplicate using the `check_duplicate_payment` tool.
3. If the tool identifies a duplicate, automatically reject the transaction and state the reason.
4. If not a duplicate, continue with normal evaluation:
    - If it seems suspicious (e.g., amount > $10,000, vendor is unusual, memo is vague), ask the human agent for approval.
    - Otherwise, approve it automatically.

Provide clear explanations for your decisions, especially for duplicates or suspicious transactions.
When all transactions are processed, clearly state that all transactions are processed and summarize the results. Then say 'Handing over for final report.' to signal completion.
"""

# Define the system message for the summary agent (to include reason for rejection)
summary_system_message_tools = """
You are a financial summary assistant. You will be given transaction details and their approval status.
Your task is to summarize the results of the transactions processed by the finance bot.
Generate a markdown table with the following columns:
- Vendor
- Memo
- Amount
- Status (Approved/Rejected)
- Reason (especially note if rejected due to being a duplicate)

The summary should include the total number of transactions, the number of approved transactions, and the number of rejected transactions.
The summary should be concise and clear.
Once you've generated the summary, append the following marker on a new line:
==== SUMMARY GENERATED ====
"""

# Create agents
with llm_config_openai:
    finance_bot_tools = ConversableAgent(
        name="finance_bot_tools",
        system_message=finance_system_message_tools,
        functions=[check_duplicate_payment],  # Register the tool
    )
    summary_bot_tools = ConversableAgent(
        name="summary_bot_tools",
        system_message=summary_system_message_tools,
    )

human_agent_tools = ConversableAgent(
    name="human_tools",
    human_input_mode="ALWAYS",
)

def is_termination_msg_tools(msg: dict[str, Any]) -> bool:
    content = msg.get("content", "")
    return (content is not None) and "==== SUMMARY GENERATED ====" in content

# Generate new transactions including potential duplicates
transactions_tools = [
    "Transaction: $500.00 to Staples. Memo: Quarterly supplies.",  # Potential duplicate
    "Transaction: $4000.00 to Unicorn LLC. Memo: Reimbursement.",
    "Transaction: $12000.00 to Globex. Memo: Confidential.",  # Potential duplicate
    "Transaction: $22000.00 to Initech. Memo: Urgent request."
]
initial_prompt_tools = (
    "Please process the following transactions, checking for duplicates:\n\n" +
    "\n".join([f"{i+1}. {tx}" for i, tx in enumerate(transactions_tools)])
)

print("--- Initial Prompt for Group Chat with Tools ---")
print(initial_prompt_tools)
print("----------------------------------------------")

pattern_tools = AutoPattern(
    initial_agent=finance_bot_tools,
    agents=[finance_bot_tools, summary_bot_tools],
    user_agent=human_agent_tools,
    group_manager_args={
        "llm_config": llm_config_openai,
        "is_termination_msg": is_termination_msg_tools
    },
)

result_tools, _, _ = initiate_group_chat(
    pattern=pattern_tools,
    messages=initial_prompt_tools,
)

print("\n--- Group Chat with Tools: Summary ---")
print(result_tools.summary)
print("\n--- Group Chat with Tools: Last Message ---")
if result_tools.chat_history:
    print(result_tools.chat_history[-1]["content"])
else:
    print("No chat history recorded.")

--- Initial Prompt for Group Chat with Tools ---
Please process the following transactions, checking for duplicates:

1. Transaction: $500.00 to Staples. Memo: Quarterly supplies.
2. Transaction: $4000.00 to Unicorn LLC. Memo: Reimbursement.
3. Transaction: $12000.00 to Globex. Memo: Confidential.
4. Transaction: $22000.00 to Initech. Memo: Urgent request.
----------------------------------------------
human_tools (to chat_manager):

Please process the following transactions, checking for duplicates:

1. Transaction: $500.00 to Staples. Memo: Quarterly supplies.
2. Transaction: $4000.00 to Unicorn LLC. Memo: Reimbursement.
3. Transaction: $12000.00 to Globex. Memo: Confidential.
4. Transaction: $22000.00 to Initech. Memo: Urgent request.

--------------------------------------------------------------------------------

Next speaker: finance_bot_tools


>>>>>>>> USING AUTO REPLY...
finance_bot_tools (to chat_manager):

***** Suggested tool call (call_AFZeEAZmET7hcRTPwtWgxWPo): check_dup

## 7. Structured Outputs: Ensuring Consistent Responses

To ensure our `summary_bot` produces reports in a consistent, predictable JSON format for easy integration with other systems, we'll use structured outputs with Pydantic models.

**Note**: Structured output support depends on the LLM provider and model. OpenAI's `gpt-4o-mini` should support this well.

In [ ]:
# Define Pydantic models for structured output
class TransactionAuditEntry(BaseModel):
    vendor: str
    amount: float
    memo: str
    status: str  # e.g., "Approved", "Rejected"
    reason: str

class AuditLogSummary(BaseModel):
    total_transactions: int
    approved_count: int
    rejected_count: int
    transactions: List[TransactionAuditEntry]
    summary_generated_time: str # ISO 8601 timestamp

# Configure a new LLMConfig for the summary bot, specifying the response_format
summary_llm_config_structured = LLMConfig(
    api_type="openai",
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
    temperature=0.2,
    response_format=AuditLogSummary,  # Specify the Pydantic model here
)

# Define the system message for the summary agent (simpler, no formatting instructions)
summary_system_message_structured = """
You are a financial summary assistant that generates audit logs in a specific JSON format.
Analyze the transaction details and their approval status from the conversation history.
Include each transaction with its vendor, amount, memo, status, and reason.
Also include a 'summary_generated_time' field with the current ISO 8601 timestamp.
"""

# Create the summary agent with structured output config
with summary_llm_config_structured: # This applies the config to summary_bot_structured
    summary_bot_structured = ConversableAgent(
        name="summary_bot_structured",
        system_message=summary_system_message_structured,
    )

# Finance bot and human agent can remain the same as in the 'Tools' section
# We'll reuse finance_bot_tools and human_agent_tools
with llm_config_openai: # For finance_bot_tools, which doesn't need structured output for its own replies
    finance_bot_structured = ConversableAgent(
        name="finance_bot_structured",
        system_message=finance_system_message_tools, # Reusing system message from tools section
        functions=[check_duplicate_payment],
    )

human_agent_structured = ConversableAgent(
    name="human_structured",
    human_input_mode="ALWAYS",
)

# Update termination message to check for structured output (presence of a key field)
def is_termination_msg_structured(msg: dict[str, Any]) -> bool:
    content = msg.get("content", "")
    if not content:
        return False
    try:
        # Try to parse the content as JSON (structured output)
        data = json.loads(content)
        return isinstance(data, dict) and "summary_generated_time" in data
    except json.JSONDecodeError:
        # Fallback for safety, though ideally summary_bot_structured always outputs JSON
        return "==== SUMMARY GENERATED ====" in content

print("Agents and Pydantic models for structured output configured.")

Agents and Pydantic models for structured output configured.


In [ ]:
# Generate new transactions
transactions_structured = [
    "Transaction: $500.00 to Staples. Memo: Quarterly supplies.",
    "Transaction: $4000.00 to Unicorn LLC. Memo: Reimbursement.",
    "Transaction: $12000.00 to Globex. Memo: Confidential.",
    "Transaction: $22000.00 to Initech. Memo: Urgent request."
]
initial_prompt_structured = (
    "Please process the following transactions, checking for duplicates, and provide a structured audit log:\n\n" +
    "\n".join([f"{i+1}. {tx}" for i, tx in enumerate(transactions_structured)])
)

print("--- Initial Prompt for Group Chat with Structured Output ---")
print(initial_prompt_structured)
print("----------------------------------------------------------")

pattern_structured = AutoPattern(
    initial_agent=finance_bot_structured,
    agents=[finance_bot_structured, summary_bot_structured],
    user_agent=human_agent_structured,
    group_manager_args={
        "llm_config": llm_config_openai, # Group manager uses standard config
        "is_termination_msg": is_termination_msg_structured
    },
)

result_structured, _, _ = initiate_group_chat(
    pattern=pattern_structured,
    messages=initial_prompt_structured,
)

print("\n--- Group Chat with Structured Output: Summary ---")
print(result_structured.summary)

print("\n--- Group Chat with Structured Output: Final Structured Message ---")
if result_structured.chat_history:
    final_message_content = result_structured.chat_history[-1]["content"]
    print("Raw final message content:")
    print(final_message_content)
    try:
        structured_data = json.loads(final_message_content)
        print("\nParsed structured data (JSON):")
        print(json.dumps(structured_data, indent=4))
        # You can now work with structured_data as a Python dictionary
        # For example, to validate with Pydantic model:
        audit_summary_obj = AuditLogSummary(**structured_data)
        print("\nPydantic model loaded successfully:", audit_summary_obj.total_transactions, "transactions")
    except json.JSONDecodeError:
        print("\nFinal message was not valid JSON.")
    except Exception as e:
        print(f"\nError processing final message: {e}")
else:
    print("No chat history recorded.")

--- Initial Prompt for Group Chat with Structured Output ---
Please process the following transactions, checking for duplicates, and provide a structured audit log:

1. Transaction: $500.00 to Staples. Memo: Quarterly supplies.
2. Transaction: $4000.00 to Unicorn LLC. Memo: Reimbursement.
3. Transaction: $12000.00 to Globex. Memo: Confidential.
4. Transaction: $22000.00 to Initech. Memo: Urgent request.
----------------------------------------------------------
human_structured (to chat_manager):

Please process the following transactions, checking for duplicates, and provide a structured audit log:

1. Transaction: $500.00 to Staples. Memo: Quarterly supplies.
2. Transaction: $4000.00 to Unicorn LLC. Memo: Reimbursement.
3. Transaction: $12000.00 to Globex. Memo: Confidential.
4. Transaction: $22000.00 to Initech. Memo: Urgent request.

--------------------------------------------------------------------------------

Next speaker: finance_bot_structured


>>>>>>>> USING AUTO REPLY...


## 8. Conclusion

Congratulations! You've walked through building a multi-agent financial compliance system using AG2, covering:

1.  **LLM Configuration**: Setting up the model (`gpt-4o-mini`).
2.  **ConversableAgent**: Creating basic intelligent agents.
3.  **Human in the Loop**: Incorporating human oversight for critical decisions.
4.  **Agent Orchestration**: Using Group Chat (`AutoPattern`) to coordinate specialized agents.
5.  **Tools**: Extending agent capabilities with custom functions (like duplicate payment detection).
6.  **Structured Outputs**: Ensuring consistent and reliable output formats using Pydantic models for system integration.

This tutorial provides a foundation for building more complex and powerful multi-agent applications with AutoGen (AG2). We encourage you to experiment with different models, tools, agent configurations, and orchestration patterns to suit your specific needs.